In [1]:
%cd Utils/

C:\Users\pasqu\Progetto_dataset_adult\Utils


*Importiamo le librerie necessarie*

In [2]:
#Librerie DS
import numpy as np
import pandas as pd

#Librerie proprie
from Import_e_info import *
from Trattamento_dati_mancanti import *

**Caricamento**

In [3]:
dataset = caricamento_dataset("../Datasets/adult.data")

In [4]:
dataset

,Age,Tipo_lavoro,fnlwgt,Istruzione,Anni_istruzione,Stato_sociale,Occupazione,Condizione_familiare,Etnia,Genere,Cap_guadagnato,Cap_perso,Ore_settimanali,Paese_nativo,Reddito
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


##### Osserviamo l'associazione delle variabili al target #####

In [5]:
test_chiquadro(dataset,dataset.Reddito, [dataset.Tipo_lavoro, dataset.Occupazione, dataset.Paese_nativo],
              index = ["Tipo_lavoro","Occupazione", "Paese_nativo"])

,chi2,p_value,degree of freedom,sign_0.05
Tipo_lavoro,827.718359,1.933848e-174,7,True
Occupazione,3744.898758,0.000000e+00,13,True
Paese_nativo,317.087663,8.280446e-45,40,True


##### Sono tutte significativamente dipendenti per un livello dello 0.05, per cui procediamo all'imputazione #####

*Tipo_lavoro*
###### Procediamo ad un'imputazione proporzionale dei missing values  ######

In [6]:
print("""Il numero di osservazioni per cui non conosciamo il tipo di lavoro praticato è pari a {:.3f}"""\
      .format(len(dataset[dataset.Tipo_lavoro.isna()]["Reddito"])/len(dataset)))

Il numero di osservazioni per cui non conosciamo il tipo di lavoro praticato è pari a 0.056


In [7]:
dataset[dataset.Tipo_lavoro.isna()]["Reddito"].value_counts()

 <=50K    1645
 >50K      191
Name: Reddito, dtype: int64

*Il valore percentuale delle osservazioni per cui non conosciamo la categoria del tipo di lavoro è molto
simile al tasso di disoccupazione americano del 1994. Data che l'alta quantità di tali osservazioni 
presenta un reddito minore di 50k, l'idea è sostituire a tali osservazione la condizione di disoccupati*

In [8]:
dataset[dataset.Tipo_lavoro.isna()] = dataset.fillna({"Tipo_lavoro": " Non occupied"})

##### Occupazione #####
*Potrebbe darci le stesse informazioni della variabile "Tipo_lavoro", per cui prima di procedere facciamo un test del chi2*

In [9]:
associazione_tipo_occ = test_chiquadro(dataset, dataset.Tipo_lavoro, [dataset.Occupazione], 
                                       index = ["occupazione"] )

In [10]:
associazione_tipo_occ

,chi2,p_value,degree of freedom,sign_0.05
occupazione,8599.689028,0.0,78,True


*Dato il p-value pari a 0.0 possiamo afferamre che le due variabili siano associate per cui possiamo eliminare la variabile Occupazione*

In [11]:
dataset.drop("Occupazione", axis = 1, inplace = True)

*Paese_Nativo*

In [12]:
dataset.Paese_nativo.value_counts() 

 United-States                 29170
 Mexico                          643
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 France                           29
 

###### E' possibile notare:
* Una grossa quantità di categorie 
* Una disparità tra la categoria US e le altre 

##### Giungiamo alla conclusione di creare soli due categorie: United States e no United States


In [13]:
#Creiamo una lista di valori 
Paese_Nativo = []
for i in range(len(dataset)):
    if dataset["Paese_nativo"][i] == " United-States":
        Paese_Nativo.append(" United-States")
    elif dataset["Paese_nativo"].isna()[i]:
        Paese_Nativo.append(np.nan)
    else:
        Paese_Nativo.append(" No United-States")

In [14]:
#Aggiungiamo la lista di valori al dataset 
dataset["Paese_Nativo_cat"] = Paese_Nativo

In [15]:
#Cancelliamo quella precedente
dataset.drop("Paese_nativo",axis = 1, inplace = True)

In [16]:
#Imputiamo i dati mancanti allo stesso modo di quanto fatto prima 
dataset = imputazione_proporzionale(dataset, "Paese_Nativo_cat")

Il 91.22% delle osservazioni appartiene alla categoria " United-States" 
Il 8.78% delle osservazioni appartiene alla categoria " No United-States" 


In [17]:
dataset.isna().sum() 
#Il dataset non ha più dati mancanti

Age                     0
Tipo_lavoro             0
fnlwgt                  0
Istruzione              0
Anni_istruzione         0
Stato_sociale           0
Condizione_familiare    0
Etnia                   0
Genere                  0
Cap_guadagnato          0
Cap_perso               0
Ore_settimanali         0
Reddito                 0
Paese_Nativo_cat        0
dtype: int64

In [18]:
#%cd ../Datasets
#dataset.to_csv("dataset_concat_disocc")